# Масликов Дмитрий МСМТ231

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, roc_auc_score, mean_absolute_error
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

## Чтение данных + разбиение на train, test

In [212]:
train = pd.read_csv('trainLab02.csv')

In [213]:
train

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [214]:
# Названия всех столбцов кроме target и столбцов с id
features = [col for col in train.columns if col not in ['row_id', 'time_id', 'target']]
print(features)

['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap']


In [215]:
# Удаление строк с "пустыми" ячейками в столбце target
train = train.dropna(subset=["target"])
train.reset_index(drop=True, inplace=True)

In [216]:
# Разбиение на X и y(target)
X = train[features].copy(deep=True)
y = train['target'].copy(deep=True)

In [217]:
X

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237887,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328
5237888,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819
5237889,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797
5237890,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008


In [274]:
number_of_days = len(X.date_id.unique()) # There are 481 trading days, we'll take the last n as testing set
testing_days = 10
training_days = number_of_days - testing_days
# training_days, testing_days

# mask to grab the days for training and testing
training_mask = X.date_id <= training_days
testing_mask = X.date_id > training_days

# subset and make training and validation sets
X_train_full, X_val_full, y_train_full, y_val_full = X[training_mask], X[testing_mask], y[training_mask], y[testing_mask] #train_test_split(X, y, test_size=0.20, random_state=0, shuffle=True, stratify=X['stock_id']) # random_state=8
X_train_full.shape, X_val_full.shape, y_train_full.shape, y_val_full.shape

((5138892, 14), (99000, 14), (5138892,), (99000,))

In [218]:
testing_days = 10
training_days = 50

training_mask = X.date_id <= training_days
testing_mask = (X.date_id > training_days) & (X.date_id <= training_days +  testing_days)

X_train, X_val, y_train, y_val = X[training_mask], X[testing_mask], y[training_mask], y[testing_mask] #train_test_split(X, y, test_size=0.20, random_state=0, shuffle=True, stratify=X['stock_id']) # random_state=8
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((539849, 14), (106810, 14), (539849,), (106810,))

In [219]:
train_date_id = X_train.date_id.copy(deep=True)
val_date_id = X_val.date_id.copy(deep=True)

In [220]:
# Преобразование данных
def feature_engineering(X):

    _X = X.copy()

    # features taken from https://www.kaggle.com/code/zulqarnainali/explained-singel-model-optiver
    _X["liquidity_imbalance"] = _X.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    _X["matched_imbalance"] = _X.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    _X["price_spread"] = _X["ask_price"] - _X["bid_price"]
    _X['market_urgency'] = _X['price_spread'] * _X['liquidity_imbalance']
    
    return _X.drop(columns=['date_id'])

# apply feature engineering
X_train = feature_engineering(X_train)
X_val = feature_engineering(X_val)

# X_train.shape, X_val.shape

In [277]:
X_train_full = feature_engineering(X_train_full)
X_val_full = feature_engineering(X_val_full)

In [221]:
# Convert all numerics to float32 to reduce memory footprint
X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)

## Random Forest

In [126]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.model_selection import GroupKFold, cross_val_score
from functools import partial

In [222]:
rfm = RandomForestRegressor(n_estimators=10, max_depth=2, min_samples_split=20)

In [223]:
rfm.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

RandomForestRegressor не может работать со значением NaN.
Попробуем что-то с этим сделать.

### SimpleImputer

In [227]:
imputer = SimpleImputer(strategy='mean')
X_train_si = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_val_si = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns)

# X_train_si.shape, X_val_si.shape

In [281]:
X_train_full_si = pd.DataFrame(imputer.transform(X_train_full), columns=X_train_full.columns)
X_val_full_si = pd.DataFrame(imputer.transform(X_val_full), columns=X_val_full.columns)

In [273]:
X_train_si.shape, X_val_si.shape

((539849, 17), (106810, 17))

In [228]:
X_train_si.isna().sum()

stock_id                   0
seconds_in_bucket          0
imbalance_size             0
imbalance_buy_sell_flag    0
reference_price            0
matched_size               0
far_price                  0
near_price                 0
bid_price                  0
bid_size                   0
ask_price                  0
ask_size                   0
wap                        0
liquidity_imbalance        0
matched_imbalance          0
price_spread               0
market_urgency             0
dtype: int64

In [229]:
def objective(params):
    integer_params = ['n_estimators', 'max_depth']
    for param in integer_params:
        params[param] = int(params[param])
        
    gkf = GroupKFold(n_splits=5)
    
    score = cross_val_score(estimator=RandomForestRegressor(n_estimators=int(params['n_estimators']),
                                                            max_depth=int(params['max_depth']),
                                                            random_state=123456, n_jobs=-1),
                            X=X_train_si, y=y_train,
                            scoring='neg_mean_absolute_error',
                            cv=gkf,
                            groups=train_date_id
                           )
    
    return {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}

#     model = RandomForestRegressor(n_estimators=int(params['n_estimators']),
#                                   max_depth=int(params['max_depth']),
#                                   random_state=123456, n_jobs=-1)
    
#     model.fit(X_train_si, y_train)
#     pred_val = model.predict(X_val_si)
#     score_val = mean_absolute_error(y_val, pred_val)
    
#     return score_val


space = {
    'n_estimators':hp.uniform('n_estimators',1,100),
    'max_depth':hp.uniform('max_depth',2,10),
}

trial=Trials()
# Запуск оптимизации
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            trials=trial,
            max_evals=10)

100%|██████████| 10/10 [19:48<00:00, 118.83s/trial, best loss: 5.203917403872486]


In [3]:
space = {
    'n_estimators':hp.uniform('n_estimators',1,100),
    'max_depth':hp.uniform('max_depth',2,10),
}

In [6]:
space['max_depth']

In [230]:
best

{'max_depth': 9.587097217804487, 'n_estimators': 38.9037443998886}

In [234]:
rf_params = {'max_depth': 9,
             'n_estimators': 39,
             'random_state':123456}
rf_params

{'max_depth': 9, 'n_estimators': 39, 'random_state': 123456}

In [235]:
rf_model = RandomForestRegressor(**rf_params, n_jobs=-1)
rf_model.fit(X_train_full_si, y_train_full)

RandomForestRegressor(max_depth=9, n_estimators=39, n_jobs=-1,
                      random_state=123456)

In [236]:
y_val_pred_si = rf_model.predict(X_val_full_si)

mae = mean_absolute_error(y_val_full, y_val_pred_si)
print(f"MAE on validation set: {mae}")
# 4.885491155850541 with depth=10

MAE on validation set: 4.885522412624567


### XGBoost

In [110]:
from xgboost import XGBRegressor

In [237]:
def xgb_objective(params):
    integer_params = ['n_estimators', 'max_depth']
    for param in integer_params:
        params[param] = int(params[param])
        
    gkf = GroupKFold(n_splits=5)
    
    model = XGBRegressor(objective='reg:squarederror',
                         n_estimators=int(params['n_estimators']),
                         max_depth=int(params['max_depth']),
                         random_state=123456, n_jobs=-1)
    
    score = cross_val_score(estimator=model,
                            X=X_train_si, y=y_train,
                            scoring='neg_mean_absolute_error',
                            cv=gkf,
                            groups=train_date_id
                           )
    
    return {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}

space = {
    'n_estimators':hp.uniform('n_estimators',1,100),
    'max_depth':hp.uniform('max_depth',2,10),
}

trial=Trials()
# Запуск оптимизации
best_xgb = fmin(fn=xgb_objective,
            space=space,
            algo=tpe.suggest,
            trials=trial,
            max_evals=20)  # Количество итераций оптимизации

100%|██████████| 20/20 [02:22<00:00,  7.10s/trial, best loss: 5.196506096012017]


In [238]:
best_xgb

{'max_depth': 3.6334892785521644, 'n_estimators': 15.511094118355476}

In [248]:
xgb_params = {'max_depth': 3,
             'n_estimators': 15,
             'random_state':123456}
xgb_params

{'max_depth': 3, 'n_estimators': 15, 'random_state': 123456}

In [249]:
xgb_model = XGBRegressor(**xgb_params, n_jobs=-1)
xgb_model.fit(X_train_full_si, y_train_full)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=15, n_jobs=-1,
             num_parallel_tree=None, random_state=123456, ...)

In [250]:
y_val_pred_xgb = xgb_model.predict(X_val_full_si)

mae_XGB = mean_absolute_error(y_val_full, y_val_pred_xgb)
print(f"MAE_XGB on validation set: {mae_XGB}")
# 4.87958733395043 4 15
# 4.87988595677448 4 16
# 4.879721764811808 3 16
# 4.879503682284063 3 15

MAE_XGB on validation set: 4.879503682284063


### Stacking

In [257]:
from sklearn.ensemble import StackingRegressor

In [265]:
lin = LinearRegression()

In [267]:
model1 = RandomForestRegressor(**rf_params, n_jobs=-1)
model1.fit(X_train_full_si, y_train_full)

RandomForestRegressor(max_depth=9, n_estimators=39, n_jobs=-1,
                      random_state=123456)

In [268]:
model2 = XGBRegressor(**xgb_params, n_jobs=-1)
model2.fit(X_train_full_si, y_train_full)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=15, n_jobs=-1,
             num_parallel_tree=None, random_state=123456, ...)

In [269]:
estimators = [('fr', model1), ('xgb', model2)]
st_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression(), cv=5)

In [270]:
st_model.fit(X_train_full_si, y_train_full)
pred_st_val = st_model.predict(X_val_full_si)
mean_absolute_error(y_val_full, pred_st_val)

4.877633593312249